In [1]:
import uproot3 as uproot
import uproot3_methods.classes.TLorentzVector as LVepm
import uproot3_methods.classes.TVector2 as LV2
import matplotlib.pyplot as plt
from matplotlib import gridspec
import infofile 
import numpy as np
import mplhep as hep
import awkward as ak

In [14]:
# Using data with truth values
# Arrays to hold events
num_events_precut = []
num_events_cut1 = []
num_events_cut2 = []
num_events_cut3 = []
num_events_cut4 = []

sample_names = []
files = ["ttbar_reco_dilep_neutrinos.root"]

for file in files:
    sample_name = file.split(".")[0] # file is a string, splits it at ., and takes entry at index 1.
    print("Sample name: ", sample_name) 
    
    sample_names.append(sample_name)
    print("Sample names: ", sample_names)
        
    tree = uproot.open(file)["nominal"]
    
    #met_met, met_phi,  lep0_pt, lep0_eta, lep0_phi, lep0_e, lep1_pt, lep1_eta, lep1_phi, lep1_e, b0_pt, b0_eta, b0_phi, b0_e, b1_pt, b1_eta, b1_phi, b1_e= tree.arrays(["met_met", "met_phi", "lep0_pt", "lep0_eta", "lep0_phi", "lep0_e", "lep1_pt", "lep1_eta", "lep1_phi", "lep1_e", "b0_pt", "b0_eta", "b0_phi", "b0_e", "b1_pt", "b1_eta", "b1_phi", "b1_e"], outputtype=tuple)
    met_met, met_phi, lep0_pt, lep0_eta, lep0_phi, lep0_e, lep1_pt, lep1_eta, lep1_phi, lep1_e, b0_pt, b0_eta, b0_phi, b0_e, b1_pt, b1_eta, b1_phi, b1_e, v0_pt_truth, v0_eta_truth, v0_phi_truth, v0_m_truth, v1_pt_truth, v1_eta_truth, v1_phi_truth, v1_m_truth = tree.arrays(["met_met", "met_phi", "lep0_pt", "lep0_eta", "lep0_phi", "lep0_e", "lep1_pt", "lep1_eta", "lep1_phi", "lep1_e", "b0_pt", "b0_eta", "b0_phi", "b0_e", "b1_pt", "b1_eta", "b1_phi", "b1_e", "v0_pt_truth", "v0_eta_truth", "v0_phi_truth", "v0_m_truth", "v1_pt_truth", "v1_eta_truth", "v1_phi_truth", 'v1_m_truth'], outputtype=tuple)

    print("File has been successfully opened!")
    
    lep0 = LVepm.TLorentzVectorArray.from_ptetaphi(lep0_pt, lep0_eta, lep0_phi, lep0_e)
    lep1 = LVepm.TLorentzVectorArray.from_ptetaphi(lep1_pt, lep1_eta, lep1_phi, lep1_e)
    b0 = LVepm.TLorentzVectorArray.from_ptetaphi(b0_pt, b0_eta, b0_phi, b0_e)
    b1 = LVepm.TLorentzVectorArray.from_ptetaphi(b1_pt, b1_eta, b1_phi, b1_e)
    v0_truth = LVepm.TLorentzVectorArray.from_ptetaphim(v0_phi_truth, v0_eta_truth, v0_phi_truth, v0_m_truth)
    v1_truth = LVepm.TLorentzVectorArray.from_ptetaphim(v1_phi_truth, v1_eta_truth, v1_phi_truth, v1_m_truth)
    

system_array = [lep0, lep1, b0, b1, met_met, met_phi, v0_truth, v1_truth]
nEvents = len(lep0)
print(nEvents)

event = [lep0[0], lep1[0], b0[0], b1[0], met_met[0], met_phi[0], v0_truth[0], v1_truth[0]]
print(len(event))

Sample name:  ttbar_reco_dilep_neutrinos
Sample names:  ['ttbar_reco_dilep_neutrinos']
File has been successfully opened!
10000
8


In [32]:
# Classes and functions necessary functions
class Solution:
    def __init__(self):
        self.m_v1 = None
        self.m_v2 = None
        self.m_solutions = None
        
    def __init__(self, num):
        self.m_v1 = None
        self.m_v2 = None
        self.m_solutions = num
    
#    def __init__(self, v1, v2, num):
#        self.m_v1 = v1
#        self.m_v2 = v2
#        self.m_solutions = num

    def setSolutions1(self,num, a, b):
        self.m_v1=a
        self.m_v2=b
        self.m_solutions=num

    def setSolutions2(self,num):
        self.m_solutions=num

    def getNumSolutions(self):
        return self.m_solutions

    def getv1(self):
        return self.m_v1

    def getv2(self):
        return self.m_v2

def solveForNeutrinoEta(lepton, bJet, topmass, index, neutrinos):
    
    print("Eta = ", neutrinos[index][0])
    
    m_wmass = 80400
    Wmass2 = m_wmass * m_wmass
    m_bmass = 4180
    bmass = m_bmass

    # What do these calculate? 
    Elprime = lepton.E * neutrinos[index][2] - lepton.p3.z * neutrinos[index][1]
    print('Elprime =', Elprime)
    Ebprime = bJet.E * neutrinos[index][2] - bJet.p3.z * neutrinos[index][1]
    print('Ebprime =', Ebprime)

    A = (lepton.p3.y * Ebprime - bJet.p3.y * Elprime) / (bJet.p3.x * Elprime - lepton.p3.x * Ebprime)
    B = (Elprime * (topMass * topMass - Wmass2 - bmass * bmass - 2. *(lepton.p3.dot(bJet.p3) - lepton.E*bJet.E)) - Ebprime * Wmass2) /(2. * (lepton.p3.x * Ebprime - bJet.p3.x * Elprime))

    par1 = (lepton.p3.x * A + lepton.p3.y) / Elprime
    C = A * A + 1. - par1 * par1
    par2 = (Wmass2 / 2. + lepton.p3.x * B) / Elprime
    D = 2. * (A * B - par2 * par1)
    F = B * B - par2 * par2
    det = D * D - 4. * C * F

    sol = Solution(0)

    if det>0:
    
        tmp = np.sqrt(det) / (2. * C)
        py1 = -D / (2. * C) + tmp
        py2 = -D / (2. * C) - tmp
        px1 = A * py1 + B
        px2 = A * py2 + B
        pT2_1 = px1 * px1 + py1 * py1
        pT2_2 = px2 * px2 + py2 * py2
        pz1 = np.sqrt(pT2_1) * neutrinos[index][1]
        pz2 = np.sqrt(pT2_2) * neutrinos[index][1]

        a1=LVepm.TLorentzVector.from_xyzm(px1, py1, pz1, 0)
        a2=LVepm.TLorentzVector.from_xyzm(px2, py2, pz2, 0)
        sol.setSolutions1(2, a1, a2);

    return sol

def neutrino_weight(neutrino1, neutrino2, met_ex, met_ey):
    # where do these sigma values come from?
    sigmax = 55643.82942235848
    sigmay = 54224.08180662581
    dx = met_ex - neutrino1.p3.x - neutrino2.p3.x
    dy = met_ey - neutrino1.p3.y - neutrino2.p3.y

    return np.exp(-dx * dx / (2. * sigmax * sigmax) - dy * dy / (2. * sigmay * sigmay))

In [34]:
# Create neutrino array
neutrinos = np.zeros((2000,3))
etaStep = 1
index = 0 
eta=-5

while eta<5.0001:
    neutrinos[index][0]=eta
    neutrinos[index][1]=np.sinh(eta)
    neutrinos[index][2]=np.cosh(eta)
    index +=1
    eta+=etaStep
    
etaSize=index
#nEvents = num_events_cut3[0]
nEvents =1

# Arrays to hold neutrino data
v0_pt = np.zeros(nEvents)
v0_eta = np.zeros(nEvents)
v0_phi = np.zeros(nEvents)
v0_m = np.zeros(nEvents)
v0_e = np.zeros(nEvents)
v1_pt = np.zeros(nEvents)
v1_eta = np.zeros(nEvents)
v1_phi = np.zeros(nEvents)
v1_m = np.zeros(nEvents)
v1_e = np.zeros(nEvents)

# Lepton and Jet 4-Vectors
l0 = event[0]
l1 = event[1]
b0 = event[2]
b1 = event[3]

# We need MET in x- and y-directions, currently in polar coordinates
met_ex = event[4]*np.cos(event[5])
met_ey = event[4]*np.sin(event[5])

max_weight=0
topMass=172500
weight=0

# Attempt to solve eqns for different values of eta
for j in range(0, etaSize):

    # Need to define solveForNeutrinoEta
    print('Answer 1')
    ans1= solveForNeutrinoEta(l0, b0, topMass, j, neutrinos)
    print('Answer 2')
    ans2= solveForNeutrinoEta(l1, b1, topMass, j, neutrinos)
    print('Answer 3')
    ans3= solveForNeutrinoEta(l0, b1, topMass, j, neutrinos)
    print('Answer 4')
    ans4= solveForNeutrinoEta(l1, b0, topMass, j, neutrinos)

    if (ans1.getNumSolutions()>0 and ans2.getNumSolutions()>0):
        sol1 = neutrino_weight(ans1.getv1(), ans2.getv1(), met_ex, met_ey)
        sol2 = neutrino_weight(ans1.getv1(), ans2.getv2(), met_ex, met_ey)
        sol3 = neutrino_weight(ans1.getv2(), ans2.getv1(), met_ex, met_ey)
        sol4 = neutrino_weight(ans1.getv2(), ans2.getv2(), met_ex, met_ey)

    if (ans3.getNumSolutions()>0 and ans4.getNumSolutions()>0):
        sol5 = neutrino_weight(ans3.getv1(), ans4.getv1(), met_ex, met_ey)
        sol6 = neutrino_weight(ans3.getv1(), ans4.getv2(), met_ex, met_ey)
        sol7 = neutrino_weight(ans3.getv2(), ans4.getv1(), met_ex, met_ey)
        sol8 = neutrino_weight(ans3.getv2(), ans4.getv2(), met_ex, met_ey)

    # Save the solution with the highest weight
    if (ans1.getNumSolutions() > 0 and ans2.getNumSolutions()>0):
        weight=sol1
        v0 = ans1.getv1()
        v2 = ans2.getv2()

        if (sol2 > weight):
            weight = sol2
            v0 = ans1.getv1()
            v1 = ans2.getv2()

        if (sol3 > weight):
            weight = sol3
            v0 = ans1.getv2()
            v1 = ans2.getv1()

        if (sol4 > weight):
            weight = sol4
            v0 = ans2.getv2()
            v1 = ans2.getv2()

    if (ans3.getNumSolutions() > 0 and ans4.getNumSolutions() > 0):
        if (sol5 > weight):
            weight = sol5
            v0 = ans3.getv1()
            v1 = ans4.getv1()

        if (sol6 > weight):
            weight = sol6
            v0 = ans3.getv1()
            v1 = ans4.getv2()

        if (sol7 > weight):
            weight = sol7
            v0 = ans3.getv2()
            v1 = ans4.getv1()

        if (sol8 > weight):
            weight = sol8
            v0 = ans3.getv2()
            v1 = ans4.getv2()

    #Save solution with largest weight per neutrino eta
    if (weight > max_weight):
        max_weight = weight
        bv0 = v0
        bv1 = v1

# Save solution to neutrino array
i=0
if (bv0 is not None) and (bv1 is not None):
    v0_pt[i] = bv0.pt
    v0_eta[i] = bv0.eta
    v0_phi[i] = bv0.phi
    v0_e[i] = bv0.E
    v0_m[i] = 0
    v1_pt[i] = bv1.pt
    v1_eta[i] = bv1.eta
    v1_phi[i] = bv1.phi
    v1_m[i] = 0
    v1_e[i] = bv1.E

Answer 1
Eta =  -5.0
Elprime = 4243127.374381894
Ebprime = 12689702.968925945
Answer 2
Eta =  -5.0
Elprime = 5877032.941480552
Ebprime = 8935121.220771637
Answer 3
Eta =  -5.0
Elprime = 4243127.374381894
Ebprime = 8935121.220771637
Answer 4
Eta =  -5.0
Elprime = 5877032.941480552
Ebprime = 12689702.968925945
Answer 1
Eta =  -4.0
Elprime = 1561055.226568128
Ebprime = 4668399.1727213
Answer 2
Eta =  -4.0
Elprime = 2162441.0310444357
Ebprime = 3287176.016988782
Answer 3
Eta =  -4.0
Elprime = 1561055.226568128
Ebprime = 3287176.016988782
Answer 4
Eta =  -4.0
Elprime = 2162441.0310444357
Ebprime = 4668399.1727213
Answer 1
Eta =  -3.0
Elprime = 574540.8056069084
Ebprime = 1717729.7491015429
Answer 2
Eta =  -3.0
Elprime = 796608.8163886049
Ebprime = 1209634.089317353
Answer 3
Eta =  -3.0
Elprime = 574540.8056069084
Ebprime = 1209634.089317353
Answer 4
Eta =  -3.0
Elprime = 796608.8163886049
Ebprime = 1717729.7491015429
Answer 1
Eta =  -2.0
Elprime = 212070.35551821147
Ebprime = 632791.8506479

In [30]:
v0_reco_xyz = LVepm.TLorentzVector.from_ptetaphi(v0_pt[0], v0_eta[0], v0_phi[0], v0_e[0])
v0_reco = LVepm.PtEtaPhiMassLorentzVector(v0_reco_xyz.pt, v0_reco_xyz.eta, v0_reco_xyz.phi, 0)
v1_reco_xyz = LVepm.TLorentzVector.from_ptetaphi(v1_pt[0], v1_eta[0], v1_phi[0], v1_e[0])
v1_reco = LVepm.PtEtaPhiMassLorentzVector(v1_reco_xyz.pt, v1_reco_xyz.eta, v1_reco_xyz.phi, 0)

In [31]:
print('v0 Reco: ', v0_reco)
print('v0 Truth: ', v0_truth[0])

print('v1 Reco: ', v1_reco)
print('v1 Truth: ', v1_truth[0])

v0 Reco:  PtEtaPhiMassLorentzVector(pt=1.3372e+05, eta=2.1649e-15, phi=1.7709, mass=0)
v0 Truth:  PtEtaPhiMassLorentzVector(pt=-1.6359, eta=3.0414, phi=-1.6359, mass=52.107)
v1 Reco:  PtEtaPhiMassLorentzVector(pt=1.2516e+05, eta=2.1649e-15, phi=-1.8777, mass=0)
v1 Truth:  PtEtaPhiMassLorentzVector(pt=-0.89597, eta=2.7591, phi=-0.89597, mass=7.2474)
